# Data Wrangling with MongoDB

## Chosen Area

* Location: Cairo, Egypt
* Data: https://s3.amazonaws.com/metro-extracts.mapzen.com/cairo_egypt.osm.bz2

## Data Audit

In [1]:
import xml.etree.cElementTree as cET
import pprint
import re
import codecs
import json
import collections
import os
from collections import defaultdict
import string
from pymongo import MongoClient


In [2]:

datadir = "data"
datafile = "cairo_egypt.osm"
cairo_data = os.path.join(datadir, datafile)

### Exploring the data

In [3]:
#Parse through the file with ElementTree and count the number of unique element types to understand overall structure.
def count_tags(filename):
        tags = {}
        for event, elem in cET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags
cairo_tags = count_tags(cairo_data)
pprint.pprint(cairo_tags)

{'bounds': 1,
 'member': 964,
 'nd': 720267,
 'node': 553028,
 'osm': 1,
 'relation': 221,
 'tag': 253248,
 'way': 112845}


In [4]:
#people invovlved in the map editing.
def process_map(filename):
    users = set()
    for _, element in cET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
users = process_map(cairo_data)
len(users)

931

## 1. Problems Encountered in the Map

### First let's explore the street names

In [5]:

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
common_street_names = ["Street", "Road", "District"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in common_street_names:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

In [6]:
cairo_street_types = audit(cairo_data)
for key in sorted(dict(cairo_street_types))[30:40]:
    print key, dict(cairo_street_types)[key]

Al-Bakry set(['Al-Sayed Al-Bakry'])
Al-Haram set(['Al-Haram'])
Al-Hussein set(['Haret Al-Hussein'])
Al-Mansureya set(['Tariq Al-Mansureya'])
Al-Markazi set(['Al-Mehwar Al-Markazi'])
Al-Roda set(['Qalaat Al-Roda'])
Al-Sanhoory set(['Dr. Abdel Raziq Al-Sanhoory'])
Al-Sherif set(['Al-Quds Al-Sherif'])
Al-Zaheri set(['Abu Dawoud Al-Zaheri'])
Awad set(['Dr. Mohamed Awad'])


### It seems that the data for street names is too messy to be fixed. There is huge inconsistency and no certain format is followed when the address is added

### * What about the postal codes ?

In [7]:
def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
    
    elif twoDigits != 95:
        invalid_zipcodes[twoDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes



In [8]:
cairo_zipcode = audit_zip(cairo_data)
for key in sorted(dict(cairo_zipcode))[:10]:
    print key, dict(cairo_zipcode)[key]

00 set(['002'])
01 set(['01066047247'])
11 set(['11511', '11321', '11221', '11774', '11381', '11111', '11241', '11361', '11811', '11421', '11865', '11835', '11551', '11211', '11936', '11231', '11311', '11351', '11371', '11759', '11451', '11476', '11737', '11717', '11431', '11517'])
12 set(['12111', '12519', '12577', '12561', '12573', '12516', '12510', '12511', '12512', '12566', '12411', '12555', '12541', '12311', '12513', '12344'])
19 set(['19511'])
24 set(['2466'])
25 set(['2500'])
31 set(['31'])
56 set(['560037'])
76 set(['7608'])


### Many zip codes are incorrect and inconsistent. There is even a phone number in the data. So let's try to clean the data and only keep the correct postal codes.
### The correct postal code in cairo consists of 5 numbers.

In [9]:
postcode_matching = re.compile("^\d{5}$")

def audit_zipcode(invalid_zipcodes, zipcode):
    twoDigits = zipcode[0:2]
    
    if not twoDigits.isdigit():
        invalid_zipcodes[twoDigits].add(zipcode)
    
    elif twoDigits != 95:
        invalid_zipcodes[twoDigits].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in cET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes
cairo_zipcode = audit_zip(cairo_data)
for street_type, ways in cairo_zipcode.iteritems():
    for name in ways:
        if (postcode_matching.match(name)):
            better_name = name
        else: 
            better_name= "N/A"
        print name, "=>", better_name

11511 => 11511
11321 => 11321
11221 => 11221
11774 => 11774
11381 => 11381
11111 => 11111
11241 => 11241
11361 => 11361
11811 => 11811
11421 => 11421
11865 => 11865
11835 => 11835
11551 => 11551
11211 => 11211
11936 => 11936
11231 => 11231
11311 => 11311
11351 => 11351
11371 => 11371
11759 => 11759
11451 => 11451
11476 => 11476
11737 => 11737
11717 => 11717
11431 => 11431
11517 => 11517
2500 => N/A
002 => N/A
12111 => 12111
12519 => 12519
12577 => 12577
12561 => 12561
12573 => 12573
12516 => 12516
12510 => 12510
12511 => 12511
12512 => 12512
12566 => 12566
12411 => 12411
12555 => 12555
12541 => 12541
12311 => 12311
12513 => 12513
12344 => 12344
None => N/A
19511 => 19511
31 => N/A
560037 => N/A
7608 => N/A
01066047247 => N/A
2466 => N/A
شارع أحمد فهيم بيومي => N/A


## Converting the XML into JSON format for mongoDB

In [20]:
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def shape_element(element):
    node = {}
    node["created"]={}
    node["address"]={}
    node["pos"]=[]
    refs=[]
    
    # we only process the node and way tags
    if element.tag == "node" or element.tag == "way" :
        if "id" in element.attrib:
            node["id"]=element.attrib["id"]
        node["type"]=element.tag

        if "visible" in element.attrib.keys():
            node["visible"]=element.attrib["visible"]
      
        # the key-value pairs with attributes in the CREATED list are added under key "created"
        for elem in CREATED:
            if elem in element.attrib:
                node["created"][elem]=element.attrib[elem]
                
        # attributes for latitude and longitude are added to a "pos" array
        # include latitude value        
        if "lat" in element.attrib:
            node["pos"].append(float(element.attrib["lat"]))
        # include longitude value    
        if "lon" in element.attrib:
            node["pos"].append(float(element.attrib["lon"]))

        
        for tag in element.iter("tag"):
            if not(problemchars.search(tag.attrib['k'])):
                if tag.attrib['k'] == "addr:housenumber":
                    node["address"]["housenumber"]=tag.attrib['v']
                    
                if tag.attrib['k'] == "addr:postcode":
                    node["address"]["postcode"]=tag.attrib['v']
                
                if tag.attrib['k'] == "addr:street":
                    node["address"]["street"]=tag.attrib['v']
                    node["address"]["street"] = node["address"]["street"]

                
                if tag.attrib['k'].find("addr")==-1:
                    node[tag.attrib['k']]=tag.attrib['v']
                    
        for nd in element.iter("nd"):
             refs.append(nd.attrib["ref"])
                
        if node["address"] =={}:
            node.pop("address", None)

        if refs != []:
           node["node_refs"]=refs
            
        return node
    else:
        return None

# process the xml openstreetmap file, write a json out file and return a list of dictionaries
def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in cET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [21]:
# process the file
data = process_map(cairo_data, True)

In [22]:
client = MongoClient()
db = client.CairoOSM
collection = db.cairoMAP
collection.insert_many(data)

## Data Overview

###  The xml file size

In [23]:
os.path.getsize(cairo_data)/1024/1024

121L

### The JSON file size

In [24]:
os.path.getsize(os.path.join(datadir, "cairo_egypt.osm.json"))/1024/1024

190L

### The Number of documents

In [25]:
collection.find().count()

2663492

### The Number of Unique users

In [26]:
len(collection.group(["created.uid"], {}, {"count":0}, "function(o, p){p.count++}"))

925

### The Number of Nodes

In [27]:
collection.find({"type":"node"}).count()

2212092

### The Number of Ways

In [28]:
collection.find({"type":"way"}).count()

451364

### Top 5 users with most contributions

In [29]:
pipeline = [{"$group":{"_id": "$created.user",
                       "count": {"$sum": 1}}},
            {"$sort": {"count": -1}},
            {"$limit": 5}]
result = collection.aggregate(pipeline)
print(list(result))

[{u'count': 444824, u'_id': u'Salmanjk'}, {u'count': 247396, u'_id': u'Mohawow'}, {u'count': 241584, u'_id': u'warneke7'}, {u'count': 187992, u'_id': u'Triscia'}, {u'count': 149512, u'_id': u'Allegro34'}]


### Proportion of the top user contributions

In [30]:
pipeline = [{"$group":{"_id": "$created.user",
                       "count": {"$sum": 1}}},
            {"$project": {"proportion": {"$divide" :["$count",collection.find().count()]}}},
            {"$sort": {"proportion": -1}},
            {"$limit": 5}]
result = collection.aggregate(pipeline)
print(list(result))

[{u'_id': u'Salmanjk', u'proportion': 0.16700782281305895}, {u'_id': u'Mohawow', u'proportion': 0.09288407849544883}, {u'_id': u'warneke7', u'proportion': 0.09070198070803291}, {u'_id': u'Triscia', u'proportion': 0.070581026712301}, {u'_id': u'Allegro34', u'proportion': 0.05613382732142616}]


### Most popular cuisines

In [31]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant", "cuisine":{"$exists":1}}}, 
            {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},        
            {"$sort":{"count":-1}}, 
            {"$limit":10}]
result = collection.aggregate(pipeline)
print(list(result))

[{u'count': 76, u'_id': u'regional'}, {u'count': 40, u'_id': u'italian'}, {u'count': 28, u'_id': u'chicken'}, {u'count': 28, u'_id': u'pizza'}, {u'count': 28, u'_id': u'american'}, {u'count': 24, u'_id': u'burger'}, {u'count': 16, u'_id': u'kebab'}, {u'count': 16, u'_id': u'chinese'}, {u'count': 16, u'_id': u'ice_cream'}, {u'count': 12, u'_id': u'asian'}]


### Number of universities in Cairo

In [32]:
pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity": "university", "name":{"$exists":1}}},
            {"$group":{"_id":"$name", "count":{"$sum":1}}},
            {"$sort":{"count":-1}}]
result = collection.aggregate(pipeline)
len(list(result))

50

This number is way more than the number of universities in Cairo. Some faculties were entered as universities.

### Top 5 common amenities

In [33]:
pipeline = [{"$match":{"amenity":{"$exists":1}}},
            {"$group":{"_id":"$amenity", "count":{"$sum":1}}},
            {"$sort":{"count":-1}},
             {"$limit":5}]
result = collection.aggregate(pipeline)
print(list(result))

[{u'count': 1896, u'_id': u'place_of_worship'}, {u'count': 1520, u'_id': u'parking'}, {u'count': 980, u'_id': u'restaurant'}, {u'count': 820, u'_id': u'school'}, {u'count': 764, u'_id': u'cafe'}]


## Conclusion

* The data for Cairo, Egypt is messy, inconsistent and incomplete. Some features cannot even be repaired or processed.
* Most citizens do not know the postal code or even its correct format. From personal experience, I did not know my zip code till recently when I needed it.
* There is no address convention and the same address an be written in many ways (even in official papers).
* Some data are entered in different language (Arabic) specially when it is difficult to write the street name in English.

### Ideas for improvement

* The data should be verified, e.x: The zip code should consist of 5 numbers.
* The address could be distributed to many sub-fields (governorate, street, building no,..etc) to have consistent format and facilitate data cleaning and processing.
* A sample entry for each field will help users understand the required data for this field. Some users entered phone numbers in the zipcode field.
* Drop-down menus for the area names would improve the data quality. This idea is already implemented in some online food services.
* There is a census being repaired now in Egypt and the governmental data could be used which would be more accurate.

### Some challenges with applyng the ideas

* The language is a huge obstacle. Many Egyptians do not speak English and many will always find it easier to add the data in Arabic.
* Adding a predefined set in a dropmenu could cause a problem because street names are changes so oftenand the list should be updated.

### References

* https://mapzen.com/data/metro-extracts/
* https://www.openstreetmap.org/#map=10/30.0346/31.5651   